# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [163]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output
import cohere

In [164]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")

In [165]:
## bloque conexion a Cohere

co = cohere.ClientV2(api_key)

response = co.chat(
    model="command-r-plus-08-2024",
    messages=[{"role": "user", "content": "hello world!"}],
)

print(response)



id='2cdcf0bb-02b7-48ac-9ee8-eaaa3a9804db' finish_reason='COMPLETE' prompt=None message=AssistantMessageResponse(role='assistant', tool_calls=None, tool_plan=None, content=[TextAssistantMessageResponseContentItem(type='text', text='Hello! How can I help you today?')], citations=None) usage=Usage(billed_units=UsageBilledUnits(input_tokens=3.0, output_tokens=9.0, search_units=None, classifications=None), tokens=UsageTokens(input_tokens=204.0, output_tokens=9.0)) logprobs=None


In [166]:
## función para obtener la respuesta de un modelo en específico con un prompt:
def usar_modelo(modelo, msje):
    response = co.chat(
        model=modelo,
        messages=[{"role": "user", "content": msje}],
    )
    content=(response.message).content[0].text
    return content

## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [167]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

{'paciente': {'nombre': 'María González', 'edad': 45},
 'fecha_admision': '2023-08-05',
 'sintomas': ['fatiga crónica', 'dolores musculares'],
 'diagnostico': 'fibromialgia',
 'tratamiento': ['fisioterapia', 'medicamentos analgésicos']}

### Prueba 1

In [168]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [169]:
instrucciones = """
  Tu objetivo es analizar un texto médico y extraer del mismo las siguientes entidades en formato JSON. 
  Primero debes identificar el nombre y la edad del paciente, luego su fecha de admisión, luego los síntomas presentados, luego su diagnóstico y finalmente el tratamiento recomendado. 
"""

restricciones = """
  En tu respuesta, asegurate de devolver las entidades en formato JSON. No incluyas texto adicional.
  El formato de las fechas debe ser "YYYY-MM-DD".
  Si algún dato no está presente en el texto, escribe "No se especifica".
"""

ejemplo = """
  # texto a analizar
  \"\"\"La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
  Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
  La próxima consulta está programada para el 15 de septiembre.\"\"\"

  # tu respuesta debe ser:
  {
    "paciente": {
      "nombre": "María González",
      "edad": 45
    },
    "fecha_admision": "2023-08-05",
    "sintomas": [
      "fatiga crónica",
      "dolores musculares"
    ],
    "diagnostico": "fibromialgia",
    "tratamiento": [
      "fisioterapia",
      "medicamentos analgésicos"
    ]
  }
"""

In [170]:
prompt = f"""
            ### Instrucciones:
            {instrucciones}

            ### Restricciones:
            {restricciones}

            ### Ejemplo:
            {ejemplo}

            ### Texto médico a analizar:
            {text_to_analize}

            """

In [171]:
# test
llm_response = usar_modelo("command-r-plus-08-2024", prompt)
print(llm_response)

{
    "paciente": {
        "nombre": "Sofía López",
        "edad": 28
    },
    "fecha_admision": "2023-04-03",
    "sintomas": [
        "fiebre alta",
        "tos persistente"
    ],
    "diagnostico": "neumonía",
    "tratamiento": [
        "antibióticos",
        "reposo"
    ]
}


In [172]:
final_result = json.loads(llm_response)

### Prueba 2

In [173]:
text_to_analize = """Carlos Ramírez, un hombre de 54 años, ingresó al Hospital General el 15 de noviembre de 2023 tras sufrir dolores severos en el pecho y dificultad para respirar./
Después de un electrocardiograma y análisis adicionales, se diagnosticó un infarto agudo de miocardio. El médico tratante, Dr. Juan Pérez, indicó tratamiento inmediato con medicamentos anticoagulantes y hospitalización en la UCI./
Se programó un cateterismo cardíaco para el 17 de noviembre."""

prompt = f"""
            ### Instrucciones:
            {instrucciones}

            ### Ejemplo:
            {ejemplo}

            ### Texto médico a analizar:
            {text_to_analize}

            """


In [174]:
llm_response = usar_modelo("command-r-plus-08-2024", prompt)
print(llm_response)

{
    "paciente": {
        "nombre": "Carlos Ramírez",
        "edad": 54
    },
    "fecha_admision": "2023-11-15",
    "sintomas": [
        "dolores severos en el pecho",
        "dificultad para respirar"
    ],
    "diagnostico": "infarto agudo de miocardio",
    "tratamiento": [
        "medicamentos anticoagulantes",
        "hospitalización en la UCI",
        "cateterismo cardíaco"
    ]
}


In [175]:
final_result = json.loads(llm_response)

### Prueba 3

In [176]:
text_to_analize = """Martina Fernández, una niña de 6 años, fue admitida al Hospital de Niños  por presentar fiebre alta, vómitos y deshidratación severa./
El diagnóstico realizado por la Dra. Ana López determinó una gastroenteritis viral. El tratamiento recomendado incluyó hidratación intravenosa y reposición de electrolitos./
Se indicó seguimiento médico para el 22 de octubre."""

prompt = f"""
            ### Instrucciones:
            {instrucciones}

            ### Ejemplo:
            {ejemplo}

            ### Texto médico a analizar:
            {text_to_analize}

            """

In [177]:
llm_response = usar_modelo("command-r-plus-08-2024", prompt)
print(llm_response)

{
    "paciente": {
        "nombre": "Martina Fernández",
        "edad": 6
    },
    "fecha_admision": "No se especifica",
    "sintomas": [
        "fiebre alta",
        "vómitos",
        "deshidratación severa"
    ],
    "diagnostico": "gastroenteritis viral",
    "tratamiento": [
        "hidratación intravenosa",
        "reposición de electrolitos"
    ]
}


In [178]:
final_result = json.loads(llm_response)

### Prueba 4

In [179]:
text_to_analize = """Ricardo Pérez, acudió al Hospital Regional el 1 de marzo de 2023 debido a fatiga persistente, pérdida de peso y tos con sangre./
Tras diversos estudios, se le diagnosticó tuberculosis pulmonar activa. El médico encargado, Dr. Andrés Gómez, recetó un tratamiento antibiótico de larga duración, supervisado mensualmente./
El paciente deberá regresar para una evaluación el 5 de abril."""

prompt = f"""
            ### Instrucciones:
            {instrucciones}

            ### Ejemplo:
            {ejemplo}

            ### Texto médico a analizar:
            {text_to_analize}

            """

In [180]:
llm_response = usar_modelo("command-r-plus-08-2024", prompt)
print(llm_response)

{
    "paciente": {
        "nombre": "Ricardo Pérez",
        "edad": "No especificada"
    },
    "fecha_admision": "2023-03-01",
    "sintomas": [
        "fatiga persistente",
        "pérdida de peso",
        "tos con sangre"
    ],
    "diagnostico": "tuberculosis pulmonar activa",
    "tratamiento": "tratamiento antibiótico de larga duración"
}


In [181]:
final_result = json.loads(llm_response)

## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [182]:
contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}
}

In [183]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto agregado con exito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."
    
functions_map = {
    "add_contact": add_contact,
    "get_information": get_information
}

In [184]:
# defino tools para el modelo de cohere
tools = [
    {
        "type": "function",
        "function": {
            "name": "add_contact",
            "description": "Agrega un nuevo contacto al diccionario de contactos.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "El nombre completo del contacto."
                    },
                    "phone": {
                        "type": "string",
                        "description": "El número de teléfono del contacto."
                    },
                    "email": {
                        "type": "string",
                        "description": "La dirección de correo electrónico del contacto."
                    }
                },
                "required": ["name", "phone", "email"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "Recupera la información de un contacto existente en el diccionario.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "El nombre completo del que queremos información."
                    }
                },
                "required": ["name"]
            }
        }
    }
]


In [185]:
# defino el prompt que le voy a pasar al modelo y mensaje de ejemplo
instrucciones = """
    Tu objetivo es responder las solicitudes de los usuarios para agregar o recuperar información de contactos en un diccionario. Se te van a dar distintas funciones que puedes utilizar para lograr responder la consulta del usuario.
"""

mensaje = "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."

In [186]:
# defino mensaje y hago print de la respuesta del modelo.
messages=[{"role": "system", "content": instrucciones},
          {"role": "user", "content": mensaje}]

response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)

print("El modelo recomienda usar las siguientes funciones:\n")
print("Mensaje del modelo:")
print(response.message.tool_plan,"\n")
print("Funciones a usar:")
for tc in response.message.tool_calls:
    print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")

messages.append({'role': 'assistant', 'tool_calls': response.message.tool_calls, 'tool_plan': response.message.tool_plan})


El modelo recomienda usar las siguientes funciones:

Mensaje del modelo:
Voy a utilizar la función add_contact() para agregar a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com. 

Funciones a usar:
Tool name: add_contact | Parameters: {"email":"juanperez@mail.com","name":"Juan Pérez","phone":"555-1234"}


In [187]:
# veamos ahora la parte de efectivamente ejecutar las llamadas

tool_content = []
# se itera sobre las llamadas a las funciones recomendadas por el modelo
for tc in response.message.tool_calls:
    # acá llamamos a la función usando los parámetros que recomienda el modelo
    tool_result= functions_map[tc.function.name](**json.loads(tc.function.arguments))
    # guardamos en una lista
    tool_content.append(json.dumps(tool_result))
    # guardamos el resultado en el historial de mensajes para usarlo en el modelo
    messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': tool_content})

print("resultados de los llamados, esto se lo daremos al modelo luego:")
for result in tool_content:
    print(json.dumps(json.loads(result), indent=2))

resultados de los llamados, esto se lo daremos al modelo luego:
"Contacto agregado con exito."


In [188]:
# veamos si efectivamente se guardó el contacto
print(contacts)

{'Joaquin Lopez': {'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'}, 'Flavio Oncativo': {'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}, 'Juan Pérez': {'phone': '555-1234', 'email': 'juanperez@mail.com'}}


In [189]:
# construyamos la rta final del modelo
response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)

In [190]:
print("Respuesta final:")
print(response.message.content[0].text)

Respuesta final:
He agregado a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com.


In [191]:
# veamos ahora para el caso de la segunda función, get_information para recuperar la información del contacto recién agregado.
mensaje = "Me podrías decir el mail de Juan Pérez?"

# defino mensaje y hago print de la respuesta del modelo.
messages=[{"role": "system", "content": instrucciones},
          {"role": "user", "content": mensaje}]

response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)

print("El modelo recomienda usar las siguientes funciones:\n")
print("Mensaje del modelo:")
print(response.message.tool_plan,"\n")
print("Funciones a usar:")
for tc in response.message.tool_calls:
    print(f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}")

messages.append({'role': 'assistant', 'tool_calls': response.message.tool_calls, 'tool_plan': response.message.tool_plan})


El modelo recomienda usar las siguientes funciones:

Mensaje del modelo:
Voy a buscar el correo electrónico de Juan Pérez. 

Funciones a usar:
Tool name: get_information | Parameters: {"name":"Juan Pérez"}


In [192]:
# veamos ahora la parte de efectivamente ejecutar la llamada

tool_content = []
# se itera sobre las llamadas a las funciones recomendadas por el modelo
for tc in response.message.tool_calls:
    # acá llamamos a la función usando los parámetros que recomienda el modelo
    tool_result= functions_map[tc.function.name](**json.loads(tc.function.arguments))
    # guardamos en una lista
    tool_content.append(json.dumps(tool_result))
    # guardamos el resultado en el historial de mensajes para usarlo en el modelo
    messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': tool_content})

print("resultados de los llamados, esto se lo daremos al modelo luego:")
for result in tool_content:
    print(json.dumps(json.loads(result), indent=2))

resultados de los llamados, esto se lo daremos al modelo luego:
{
  "phone": "555-1234",
  "email": "juanperez@mail.com"
}


In [193]:
# construyamos la rta final del modelo
response = co.chat(
    model="command-r-plus-08-2024",
    messages=messages,
    tools=tools
)

print("Respuesta final:")
print(response.message.content[0].text)

Respuesta final:
El correo electrónico de Juan Pérez es juanperez@mail.com.


In [194]:
# automatización de este proceso:

def procesar_mensaje(mensaje, modelo, tools, functions_map):

    messages=[{"role": "system", "content": instrucciones},
              {"role": "user", "content": mensaje}]

    response = co.chat(
        model=modelo,
        messages=messages,
        tools=tools
    )

    messages.append({'role': 'assistant', 'tool_calls': response.message.tool_calls, 'tool_plan': response.message.tool_plan})

    tool_content = []
    for tc in response.message.tool_calls:
        tool_result= functions_map[tc.function.name](**json.loads(tc.function.arguments))
        tool_content.append(json.dumps(tool_result))
        messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': tool_content})

    response = co.chat(
        model=modelo,
        messages=messages,
        tools=tools
    )

    print(mensaje)
    print(response.message.content[0].text)
    

In [195]:
# probemos el código con los otros ejemplos dados en el enunciado

mensaje1 = "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
mensaje2 = "Cual es el Email de Juan Pérez.?"

procesar_mensaje(mensaje1, "command-r-plus-08-2024", tools, functions_map)
procesar_mensaje(mensaje2, "command-r-plus-08-2024", tools, functions_map)

Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com.
He guardado a Lucía Gómez en tus contactos.
Cual es el Email de Juan Pérez.?
El email de Juan Pérez es juanperez@mail.com.


In [196]:
print(contacts) #lucía gomez efectivamente está en la lista

{'Joaquin Lopez': {'tel': 15456663258, 'mail': 'Joacolocolopez@gmail.com'}, 'Flavio Oncativo': {'tel': 1545554178, 'mail': 'FOncativo@hotmail.com'}, 'Juan Pérez': {'phone': '555-1234', 'email': 'juanperez@mail.com'}, 'Lucía Gómez': {'phone': '555-5678', 'email': 'lucia.gomez@gmail.com'}}


## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [197]:
def history_answer(pregunta):
    # your code here

    # PARA CONSTRUIR RESPUESTA ---------------------------------------------------
    instrucciones_rta = """
        Sos un asistente que responde preguntas basadas en un contexto dado. Dada una pregunta y un contexto, debes responder siguiendo los siguientes lineamientos dados.
    """

    restricciones_rta = """
        - Tu respuesta debe tener relación al contexto.
        - Si una pregunta se repite, debes responder de la misma manera cada vez.
        - Tu respuesta debe ser en una oración
        - Tu respuesta debe ser en el mismo lenguaje en el que se hizo la pregunta.
        - Tu respuesta debe tener emojis que tengan relación a la respuesta y la pregunta
        - Tu respuesta siempre debe ser en tercera persona
        - Si la pregunta no tiene relación al contexto, responde "Lo siento no puedo ayudarte con eso".
        - Al final de tu respuesta, siempre debes incluir 'Hakuna Matata!'.
    """

    historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

        Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

        La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

        Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco.
    """

    prompt_rta = f"""
            ### Instrucciones:
            {instrucciones_rta}

            ### Lineamientos:
            {restricciones_rta}

            ### Contexto:
            {historia}

            """

    messages_rta=[{"role": "system", "content": prompt_rta},
                {"role": "user", "content": pregunta}]

    # return the answer

    response_rta = co.chat(
        model="command-r-plus-08-2024",
        messages=messages_rta,
    )

    respuesta_al_usuario = response_rta.message.content[0].text
    
    # PARA CHEQUEAR LENGUAJE ---------------------------------------------------

    instrucciones_leng="""
        Tu objetivo es analizar dos textos y determinar si el segundo texto está en el mismo idioma que el primero. En el caso de que el segundo texto
        no esté en el mismo idioma que el primero, deberás traducir el segundo texto al mismo idioma que el primero.

        Si no es necesario hacer una traducción, simplemente responde con el segundo texto. No incluyas ningún texto adicional en tu respuesta.
    """

    prompt_leng = f"""
            ### Instrucciones:
            {instrucciones_leng}
            """
    
    consulta=f"""
        Texto 1: {pregunta}
        Texto 2: {respuesta_al_usuario}
    """
    messages_leng=[{"role": "system", "content": prompt_leng},
                    {"role": "user", "content": consulta}]

    response_leng = co.chat(
        model="command-r-plus-08-2024",
        messages=messages_leng,
    )

    respuesta_leng = response_leng.message.content[0].text

    return respuesta_leng

In [198]:
pregunta = "¿Qué edad tenía Thomas cuando se unió a la milicia?"
print(history_answer(pregunta))

Thomas se unió a la milicia a los dieciséis años, con la determinación de un león 🦁. Hakuna Matata!


In [199]:
pregunta = "Where did Thomas work?"
print(history_answer(pregunta))

Thomas trabajaba en los campos de trigo del señor feudal. 🌾 Hakuna Matata!


In [200]:
pregunta = "Quel âge avait Thomas lorsqu'il a rejoint la milice?"
print(history_answer(pregunta))

Thomas avait seize ans 🌾 quand il a troqué sa faux pour une lance et s'est enrôlé dans la milice locale. Hakuna Matata!


In [201]:
pregunta = "Como se llama el presidente de los estados unidos"
print(history_answer(pregunta))

Lo siento, no puedo ayudarte con eso. Hakuna Matata! 😊🦁🌍


## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [202]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()

chat_history = []

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    # Aquí puedes conectar tu modelo o lógica de chatbot real

    prompt = f"""

    ### Instrucciones:
    Sos un chatbot entusiasta y positivo que ayuda a las personas con sus consultas, dando consejos útiles.

    """
    
    chat_history.append({"role": "user", "content": message})

    model_message = [{"role": "system", "content": prompt}] + chat_history

    response = co.chat(
        model="command-r-plus-08-2024",
        messages=model_message,
        max_tokens=70
    )

    model_answer = response.message.content[0].text

    #agregar rta al historial
    chat_history.append({"role": "assistant", "content": model_answer})

    return model_answer

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [204]:
print(chat_history)

[{'role': 'user', 'content': 'Hola, como estas? Mi nombre es Juan. Tengo 22 años y estudio Computación.'}, {'role': 'assistant', 'content': '¡Hola, Juan! Es un placer conocerte. Me alegra saber que estás estudiando Computación, es una carrera fascinante y con muchas oportunidades en el mundo actual. ¿Cómo va tu experiencia universitaria hasta ahora? ¿Hay algún aspecto de la computación que te apasione especialmente? Estoy aquí para charlar y'}, {'role': 'user', 'content': 'Me podrías decir brevemente cuáles son los temas centrales en computación?'}, {'role': 'assistant', 'content': '¡Claro! La carrera de Computación o Informática abarca una amplia gama de temas interesantes y fundamentales. A continuación, te menciono algunos de los temas centrales que generalmente se estudian en esta carrera:\n\n1. **Programación**: Es la base de la computación y abarca el diseño y desarrollo de software utilizando diferentes lenguajes de programación'}, {'role': 'user', 'content': 'Cómo me llamo?'}, 

### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.